## to do 

1. rename the columns use more intuitive names
1. implement usd to euro conversion for the yahoo timeseries

In [1]:
%config Completer.use_jedi = False

In [2]:
import sys

In [3]:
sys.path.append("../scr")

In [4]:
import sys
from datetime import datetime, timedelta

import altair as alt
import numpy as np
import pandas as pd
import panel as pn
import yfinance as yf
from altair import datum
from utility import (
    plot_transactions,
    plot_transactions_2,
    read_ticker_ts,
    read_transactions,
)

from datetime import date, timedelta

In [5]:
import plotly.graph_objs as go
import plotly.express as px

In [6]:
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [7]:
fln = "../data/trasaction_history_18022021.csv"
tr = read_transactions(fln) 

## data preparation

### keep only the relavant columns

In [8]:
col_to_keep = ['Action', 'Time', 'Ticker', 'No. of shares', 'Price / share', 'Exchange rate',
               'Result (EUR)', 'Currency (Price / share)', 'ISIN']

tr = tr[col_to_keep]

### rename columns

In [9]:
tr.loc[tr['Action'].str.contains("buy"), 'Action'] = 'buy'
tr.loc[tr['Action'].str.contains("sell"), 'Action'] = 'sell'

### feature engineering

use the transaction history to determine
- invested amount
- average price after each trasaction
- profit after each action 


In [10]:
def calculate_average_price(group):
    '''calculate the average price based on the invested value and floating number of shares'''

    # cumulative total in euro (buy price*share - average_price*sell)
    group['cum_total_eur'] = group['invested_eur'].cumsum()
    group['ave_price_eur'] = group['cum_total_eur']/group['cum_shares']

    return group


def calculate_return(group):
    """Update transaction time history, include calculation of average price
    """

    mask_buy = group["Action"] == 'buy'
    mask_sell = group["Action"] == 'sell'

    # determine the accumulated number of shares
    group.loc[mask_buy,'action_sign']= 1
    group.loc[mask_sell,'action_sign']= -1
    
    group['cum_shares'] = (group['No. of shares'] *
                           group['action_sign']).cumsum()

    # average price, treating all actions as buy
    group['pps_eur'] = group['Price / share'] / \
        group['Exchange rate'].astype('float')  # price per share in eur
    group['invested_eur'] = group['No. of shares']*group['pps_eur']
    group = calculate_average_price(group)

    # update the ave_price whenever a sell event occurs
    for idx, row in group[mask_sell].iterrows():
        group.loc[idx, 'invested_eur'] = -group.loc[idx,'No. of shares']*group.loc[idx-1, 'ave_price_eur']
        group.loc[idx, 'ave_price_eur'] = group.loc[idx-1, 'ave_price_eur']
        group = calculate_average_price(group)

    # determine the return for each sell event
    group.loc[mask_sell, 'profit_eur'] = group.loc[mask_sell, 'pps_eur']*group.loc[mask_sell, 'No. of shares'] + \
        group.loc[mask_sell, 'invested_eur']

    return group

In [11]:
tr=tr.loc[tr['Action'].isin(['buy','sell'])]
grouped = tr.groupby(by='Ticker')

groups = []  

for name , group in grouped:
    group.reset_index(inplace=True, drop=True)
    group = calculate_return(group)
    groups.append(group)
    
tr = pd.concat(groups).reset_index(drop=True)

### read ticker history from yfiance

In [13]:
tickers = tr[['Ticker', 'Currency (Price / share)', 'ISIN']].drop_duplicates()
tickers.rename({'Currency (Price / share)': 'Currency'}, axis=1, inplace=True)
# tickers = tickers.dropna(subset=['Ticker'])
us_tickers = tickers.loc[tickers['Currency']=='USD', 'Ticker'].to_list()
# us_tickers.append('USDEUR%3DX')

In [14]:
start = tr.Time.min()
end = tr.Time.max() +  timedelta(1)
data = yf.download(us_tickers, start, end)['Adj Close']

[*********************100%***********************]  36 of 36 completed

1 Failed download:
- LTM: No data found, symbol may be delisted


In [15]:
df_forex = yf.download(['USDEUR%3DX'], start, end)[['Adj Close']]
df_forex = df_forex.reset_index()
df_forex.rename({'Date':"date", 'Adj Close':"rate"}, axis = 1, inplace=1)

[*********************100%***********************]  1 of 1 completed


### identify tickers not downloaded

In [16]:
mask = data.isna().mean() == 1.0

In [17]:
not_found = mask.loc[mask].keys().values

In [18]:
print(f'these tickers are not included in the time history: {not_found}')

these tickers are not included in the time history: ['LTM']


### combine yahoo time history with transaction data

In [19]:
# retrieve stock value at close
def ticker_price_history(data, ticker):
    """filter time history of the specifed ticker, history of ticker close price """
    tts_sub = data[ticker]  # ticker time series
    tts_sub = tts_sub.reset_index()
#     tts_sub.columns = tts_sub.columns.droplevel()
    tts_sub.columns = ['time_ts', 'close_price']

    return tts_sub

In [20]:
def merge_histories(tr_sub, tts_sub, ticker):
    """merge dataframe based on date and time

    note: invested amount is determined based on the market price at close, rather than the transaction record
    """
    merged = tts_sub.merge(right=tr_sub, left_on='time_ts',
                           right_on='Time', how="outer")
    merged = merged.merge(df_forex, left_on = 'time_ts', right_on='date')
    merged['cum_shares'] = merged['cum_shares'].fillna(method='ffill')
    merged['value'] = merged['close_price']*merged['cum_shares']*merged['rate']
    merged['ticker'] = ticker
    merged['cum_total_eur'] = merged['cum_total_eur'].fillna(
        method='ffill')
    merged['value'] = merged['value'].fillna(
        method='ffill')
    return merged

In [21]:
# loop it through for all the tickers:
dfs = []
groups = tr.groupby(by='Ticker')
tickers = data.columns

for ticker in tickers:
    if ticker not in not_found:
        # share_no_history(tr_pivoted,ticker)
        tr_sub = groups.get_group(
            ticker)[['Time', 'Ticker', 'cum_shares', 'cum_total_eur', 'profit_eur']]
        tts_sub = ticker_price_history(data, ticker)
        df = merge_histories(tr_sub, tts_sub, ticker)

        dfs.append(df)
    else:
        print(f'{ticker} not in the database')

df_combined = pd.concat(dfs)

NameError: name 'tickers_to_drop' is not defined

## monthly transaction overview

Hereby I will create an overview of total transacations

In [ ]:
mt = (
    tr.groupby(by=[pd.Grouper(key="Time", freq="M"), "Action"])["Action"]
    .count()
    .rename("transactions")
    .reset_index()
)  # montly transaction

In [ ]:
fig = go.Figure()

# reformat the date_time to provide a monthly summary (do not show the date)
mt['mnth_yr'] = mt['Time'].apply(lambda x: x.strftime('%b-%Y'))

for action in ['buy', 'sell']:
    mt_ = mt.loc[mt['Action'] == action]
    fig.add_trace(go.Bar(
        x=mt_.mnth_yr,
        y=mt_.transactions,
        name=action,
    ))

fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text='monthly transactions overview',
                  yaxis=dict(
                      title='transaction counts',
                  ))
fig.show()

## latest portofolio

1. what are the stocks in my pf?
1. total value (weekly overview)

### pie chart of portofolio composition

In [ ]:
df_end = df_combined.loc[(df_combined['time_ts'] == end -
                          timedelta(1)) & (df_combined['cum_shares'] > 0.25)]
df_end = df_end.drop_duplicates(
    subset=['time_ts', 'Ticker'], keep='last', inplace=False, ignore_index=False)

In [ ]:
import plotly.express as px

fig = px.pie(df_end, values='value', names='ticker')
fig.show()

### line plot for portofolio ts

In [ ]:
df_combined = df_combined.drop_duplicates(
    subset=['time_ts', 'ticker'], keep='last', inplace=False, ignore_index=False)

# df_agg = df_combined.pivot_table(index='time_ts', values=[
#                                  'value', 'cum_total_eur'], aggfunc='sum').reset_index()

In [ ]:
df_agg = df_combined.pivot_table(index='time_ts', values=[
                                 'value', 'cum_total_eur', 'profit_eur'], aggfunc='sum').reset_index()

In [ ]:
df_agg['realized_profit'] = df_agg['profit_eur'].cumsum()
df_agg['floating_profit'] = df_agg['value'] - df_agg['cum_total_eur']

In [ ]:
df = px.data.stocks()

fig = px.line(df_agg, x="time_ts", y=['value','cum_total_eur'],
              hover_data={"time_ts": "|%B %d, %Y"},
              title='Overview'
              )

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(hovermode="x")

fig.show()

### brief summary

In [ ]:
last_row = df_agg.tail(1)

In [ ]:
print('A brief summary')
print("You started investing with Trading 212 on {:}.".format(
    start.strftime("%b %d, %Y")))
print("Upon {:}, you've invested {:0.2f} EUR in the market, with a floating profit of {:0.2f} EUR. Total realized profit amounts {:0.2f} EUR".format(
    end.strftime("%b %d, %Y"), 
    last_row.cum_total_eur.values[0],
    last_row.floating_profit.values[0],
    last_row.realized_profit.values[0]))

### line plot of profit and loss

In [ ]:
df = px.data.stocks()

fig = px.line(df_agg, x="time_ts", y=['floating_profit','realized_profit'],
              hover_data={"time_ts": "|%B %d, %Y"},
              title='Overview'
              )


fig.add_trace(go.Scatter(
    name="total",
    x=df_agg["time_ts"], y=df_agg['floating_profit'] + df_agg['realized_profit'],
))

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(hovermode="x")

fig.show()

## stock analysis

### correlation analysis

In [ ]:
from matplotlib import pyplot as plt
cols = df_end.Ticker.dropna().unique()
retscomp = data.loc[:,  cols]
retscomp = retscomp.pct_change()
corr = retscomp.corr()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=corr,
    x=cols,
    y=cols,
    colorscale='Hot',
    reversescale=True,
    zmax=1.0,
    zmin=0.0))

fig.update_layout(
    autosize=False,
    width=600,
    height=600,)

fig.show()

### Stocks mean and Risk calculation

In [ ]:
y=retscomp.std()

In [ ]:

fig = px.scatter(x=retscomp.mean(), y=retscomp.std(), text=cols)
fig.update_traces(textposition='top center')
fig.update_layout(
#     height=800,
    title_text='Return and Risk',
    xaxis_title="Return",
    yaxis_title="Risk",
)
fig.show()

In [ ]:
cols

## return ratio variation over time